# New Object Hierarchy

Version 3

# Cayley Table

In [1]:
import numpy as np
import pprint as pp

In [2]:
class CayleyTable:

    def __init__(self, arr):
        tmp = np.array(arr, dtype=int)
        nrows, ncols = tmp.shape
        if nrows == ncols:
            if (tmp.min() >= 0) and (tmp.max() < nrows):
                self.__order = nrows
                self.__table = tmp
            else:
                raise Exception(f"All integers must be between 0 and {nrows - 1}, inclusive.")
        else:
            raise Exception(f"Input arrays must be square; this one is {nrows}x{ncols}.")

    def __repr__(self):
        return f"{self.__class__.__name__}(\n{pp.pformat(self.__table.tolist())}\n)"

    def __str__(self):
        return f"{self.__class__.__name__}({self.__table.tolist()})"

    def __getitem__(self, tup):
        row, col = tup
        return self.__table[row][col]

    @property
    def order(self):
        return self.__order

    @property
    def table(self):
        return self.__table

    def tolist(self):
        return self.__table.tolist()

    def is_associative(self):
        indices = range(len(self.__table))
        result = True
        for a in indices:
            for b in indices:
                for c in indices:
                    ab = self.__table[a][b]
                    bc = self.__table[b][c]
                    if not (self.__table[ab][c] == self.__table[a][bc]):
                        result = False
                        break
        return result

    def is_commutative(self):
        n = self.__table.shape[0]
        result = True
        # Loop over the table's upper off-diagonal elements
        for a in range(n):
            for b in range(a + 1, n):
                if self.__table[a][b] != self.__table[b][a]:
                    result = False
                    break
        return result

    def left_identity(self):
        indices = range(len(self.__table))
        identity = None
        for x in indices:
            if all(self.__table[x][y] == y for y in indices):
                identity = x
                break
        return identity

    def right_identity(self):
        indices = range(len(self.__table))
        identity = None
        for x in indices:
            if all(self.__table[y][x] == y for y in indices):
                identity = x
                break
        return identity

    def identity(self):
        left_id = self.left_identity()
        right_id = self.right_identity()
        if (left_id is not None) and (right_id is not None):
            return left_id
        else:
            return None

    def has_inverses(self):
        if self.identity:
            row_indices, col_indices = np.where(self.__table == self.identity())
            if set(row_indices) == set(col_indices):
                if len(row_indices) == self.__order:
                    return True
                else:
                    return False
            else:
                return False
        else:
            return False

    def inverse_lookup_dict(self, identity):
        elements = range(len(self.__table))
        row_indices, col_indices = np.where(self.__table == identity)
        return {elements[elem_index]: elements[elem_inv_index]
                for (elem_index, elem_inv_index)
                in zip(row_indices, col_indices)}

    def about(self):
        table_order = str(self.order)
        is_associative = str(self.is_associative())
        is_commutative = str(self.is_commutative())
        left_id = str(self.left_identity())
        right_id = str(self.right_identity())
        id = str(self.identity())
        has_inverses = str(self.has_inverses())
        return table_order, is_associative, is_commutative, left_id, right_id, id, has_inverses

In [3]:
def about_tables(list_of_cayley_tables):
    print("   Table  Order  Associative?  Commutative?  Left Id?  Right Id?  Identity?  Inverses?")
    print('-' * 85)
    for tbl in list_of_cayley_tables:
        i = list_of_cayley_tables.index(tbl) + 1
        n, assoc, comm, lid, rid, id, invs = tbl.about()
        print(f"{i :>{6}} {n :>{6}} {assoc :>{11}} {comm :>{12}} {lid :>{12}} {rid :>{9}} {id :>{10}} {invs :>{10}}")

### Testing Cayley Tables

In [4]:
# not assoc; is comm; no identities -- the RPS magma table, above
arr1 = [[0, 1, 0], [1, 1, 2], [0, 2, 2]]

# is assoc; not comm; has identity (0) --- the S3 group table
arr2 = [[0, 1, 2, 3, 4, 5], [1, 2, 0, 5, 3, 4], [2, 0, 1, 4, 5, 3],
        [3, 4, 5, 0, 1, 2], [4, 5, 3, 2, 0, 1], [5, 3, 4, 1, 2, 0]]

# is assoc; is comm; has identity (0) --- the Z4 group table
arr3 = [[0, 1, 2, 3], [1, 2, 3, 0], [2, 3, 0, 1], [3, 0, 1, 2]]

# is assoc; is comm; has identity (0) --- powerset(3) group table
arr4 = [[0, 1, 2, 3, 4, 5, 6, 7], [1, 0, 4, 5, 2, 3, 7, 6], [2, 4, 0, 6, 1, 7, 3, 5],
        [3, 5, 6, 0, 7, 1, 2, 4], [4, 2, 1, 7, 0, 6, 5, 3], [5, 3, 7, 1, 6, 0, 4, 2],
        [6, 7, 3, 2, 5, 4, 0, 1], [7, 6, 5, 4, 3, 2, 1, 0]]

arr5 = [[0, 3, 0, 3, 0, 3], [1, 4, 1, 4, 1, 4], [2, 5, 2, 5, 2, 5],
        [3, 0, 3, 0, 3, 0], [4, 1, 4, 1, 4, 1], [5, 2, 5, 2, 5, 2]]

# is assoc; is not comm; no left id; has right id --- Smarandache Groupoid
test_arrays = [arr1, arr2, arr3, arr4, arr5]
test_cayley_tables = [CayleyTable(arr) for arr in test_arrays]

about_tables(test_cayley_tables)

   Table  Order  Associative?  Commutative?  Left Id?  Right Id?  Identity?  Inverses?
-------------------------------------------------------------------------------------
     1      3       False         True         None      None       None      False
     2      6        True        False            0         0          0       True
     3      4        True         True            0         0          0       True
     4      8        True         True            0         0          0       True
     5      6        True        False         None         0       None      False


In [5]:
ct1 = CayleyTable(arr5)
ct1

CayleyTable(
[[0, 3, 0, 3, 0, 3],
 [1, 4, 1, 4, 1, 4],
 [2, 5, 2, 5, 2, 5],
 [3, 0, 3, 0, 3, 0],
 [4, 1, 4, 1, 4, 1],
 [5, 2, 5, 2, 5, 2]]
)

In [6]:
ct1.tolist()

[[0, 3, 0, 3, 0, 3],
 [1, 4, 1, 4, 1, 4],
 [2, 5, 2, 5, 2, 5],
 [3, 0, 3, 0, 3, 0],
 [4, 1, 4, 1, 4, 1],
 [5, 2, 5, 2, 5, 2]]

In [7]:
str(ct1)

'CayleyTable([[0, 3, 0, 3, 0, 3], [1, 4, 1, 4, 1, 4], [2, 5, 2, 5, 2, 5], [3, 0, 3, 0, 3, 0], [4, 1, 4, 1, 4, 1], [5, 2, 5, 2, 5, 2]])'

# Algebras

In [8]:
import functools as fnc

#### A useful pattern

In [9]:
def get_cached_value(cached_value, accessor):
    if cached_value is None:
        cached_value = accessor()
    return cached_value

## Magma

In [35]:
class Magma:
    
    def __init__(self, elems, tbl):
        self.__elements = elems
        self.__table = CayleyTable(tbl)
        self.__is_associative = None
        self.__is_commutative = None
        self.__has_identity = None
        self.__identity = None

    def __contains__(self, element):
        return element in self.__elements

    def __getitem__(self, index):
        return self.__elements[index]

    def __repr__(self):
        return f"{self.__class__.__name__}(\n{self.__elements},\n{self.__table.tolist()}\n)"

    def __str__(self):
        return f"{self.__class__.__name__}({self.__elements}, {self.__table.tolist()})"

    @property
    def elements(self):
        return self.__elements
    
    def set_elements(self, new_elements):
        if isinstance(new_elements, list):
            self.__elements = new_elements
        elif isinstance(new_elements, dict):
            self.__elements = [new_elements[elem] for elem in self.__elements]
        return self
    
    @property
    def table(self):
        return self.__table

    def op(self, *args):
        if len(args) == 1:
            if args[0] in self.__elements:
                return args[0]
            else:
                raise ValueError(f"{args[0]} is not a valid element name")
        elif len(args) == 2:
            row = self.__elements.index(args[0])
            col = self.__elements.index(args[1])
            index = self.__table[row, col]
            return self.__elements[index]
        else:
            return fnc.reduce(lambda a, b: self.op(a, b), args)
    
    def table_with_names(self):
        return [[self.__elements[index] for index in row] for row in self.__table.tolist()]

    def is_associative(self):
        return get_cached_value(self.__is_associative, self.__table.is_associative)

    def is_commutative(self):
        return get_cached_value(self.__is_commutative, self.__table.is_commutative)

    def identity(self):
        pass

## Semigroup

In [36]:
class Semigroup(Magma):

    def __init__(self, elems, tbl):
        super().__init__(elems, tbl)
        if not self.table.is_associative():
            raise ValueError("Table does not support associativity")

## Monoid

In [37]:
class Monoid(Semigroup):

    def __init__(self, elems, tbl):
        super().__init__(elems, tbl)
        if not self.table.identity():
            raise ValueError("Table has no identity element")

## Group

In [38]:
class Group(Monoid):

    def __init__(self, elems, tbl):
        super().__init__(elems, tbl)
        if not self.table.has_inverses():
            raise ValueError("Table has insufficient inverses")

## Testing Algebras

**Testing Magma**

Rock-Paper-Scisors Magma

This magma is obviously commutative, but not associative.

See https://en.wikipedia.org/wiki/Commutative_magma

* $M = \langle \{r,p,s\}, \cdot \rangle$
* For all $x, y \in M$, if $x$ *beats* $y$, then $x \cdot y = y \cdot x = x$
* Also, for all $x \in M$, $xx = x$

In [39]:
rps = Magma(['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])
rps

Magma(
['r', 'p', 's'],
[[0, 1, 0], [1, 1, 2], [0, 2, 2]]
)

In [40]:
rps.is_associative()

False

In [41]:
rps.is_commutative()

True

In [42]:
str(rps)  # TODO: FIX THIS

"Magma(['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])"

The following demonstrates that the rps magma is non-associative:

In [43]:
ps = rps.op('p', 's')
rp = rps.op('r', 'p')

r_ps = rps.op('r', ps)
rp_s = rps.op(rp, 's')

print(f"    r(ps) = r{ps} = {r_ps}, \nbut (rp)s = {rp}s = {rp_s}")

    r(ps) = rs = r, 
but (rp)s = ps = s


For other magma examples, [see this discussion](https://math.stackexchange.com/questions/779507/can-you-give-me-some-concrete-examples-of-magmas).  Also, [see this paper on groupiods](https://arxiv.org/ftp/math/papers/0304/0304490.pdf).

#### Table and Element Accessors

In [44]:
rps.table

CayleyTable(
[[0, 1, 0], [1, 1, 2], [0, 2, 2]]
)

In [45]:
rps.elements

['r', 'p', 's']

In [46]:
rps.table_with_names()

[['r', 'p', 'r'], ['p', 'p', 's'], ['r', 's', 's']]

In [47]:
rps.table.about()

('3', 'False', 'True', 'None', 'None', 'None', 'False')

#### Magma as an Iterator and Container of Elements

In [48]:
[el for el in rps]

['r', 'p', 's']

In [49]:
'r' in rps

True

#### Replacing ("Setting") Element Names

In [50]:
full_names = ['rock', 'paper', 'scissors']
rps.set_elements(full_names)

Magma(
['rock', 'paper', 'scissors'],
[[0, 1, 0], [1, 1, 2], [0, 2, 2]]
)

In [51]:
orig_elems = ['r', 'p', 's']
mapping = dict(zip(rps.elements, orig_elems))
print(mapping)
rps.set_elements(orig_elems)

{'rock': 'r', 'paper': 'p', 'scissors': 's'}


Magma(
['r', 'p', 's'],
[[0, 1, 0], [1, 1, 2], [0, 2, 2]]
)

## Semigroups

A semigroup is an associative magma.

In [52]:
rps.is_associative()

False

The Semigroup constructor will fail if the table does not support associativity:

In [53]:
try:
    Semigroup(['r', 'p', 's'], [[0, 1, 0], [1, 1, 2], [0, 2, 2]])
except:
    print("Something went wrong")

Something went wrong


#### Smarandache Semigroup

This is Example 1.4.1 in the paper on groupoids referenced earlier.

In that reference it is called a groupoid (AKA magma) but it is associative, so that makes it a semigroup.

In [54]:
ex141_tbl = [[0, 3, 0, 3, 0, 3], [1, 4, 1, 4, 1, 4], [2, 5, 2, 5, 2, 5],
             [3, 0, 3, 0, 3, 0], [4, 1, 4, 1, 4, 1], [5, 2, 5, 2, 5, 2]]

We can make a magma out of the table.

In [55]:
ex141_magma = Magma(['a', 'b', 'c', 'd', 'e', 'f'], ex141_tbl)
ex141_magma

Magma(
['a', 'b', 'c', 'd', 'e', 'f'],
[[0, 3, 0, 3, 0, 3], [1, 4, 1, 4, 1, 4], [2, 5, 2, 5, 2, 5], [3, 0, 3, 0, 3, 0], [4, 1, 4, 1, 4, 1], [5, 2, 5, 2, 5, 2]]
)

But we can also make a semigroup out of this table, since it is associative.

In [56]:
ex141_sg = Semigroup(['a', 'b', 'c', 'd', 'e', 'f'], ex141_tbl)
ex141_sg

Semigroup(
['a', 'b', 'c', 'd', 'e', 'f'],
[[0, 3, 0, 3, 0, 3], [1, 4, 1, 4, 1, 4], [2, 5, 2, 5, 2, 5], [3, 0, 3, 0, 3, 0], [4, 1, 4, 1, 4, 1], [5, 2, 5, 2, 5, 2]]
)

We cannot make a monoid from the table, because it does not have an identity element.

In [57]:
try:
    ex141_mon = Monoid(['a', 'b', 'c', 'd', 'e', 'f'], ex141_tbl)
    ex141_mon
except:
    print("ERROR: Table has no identity element")

ERROR: Table has no identity element


### See p. 67 in Pinter for a possible example

## Monoid

A monoid is a semigroup with an identity element.

**NEED TESTS AND EXAMPLES HERE**

## Group

A group is a monoid where every element has an inverse.

TBD